## Interfaz de Texto Predictivo

---


### Equipo:
- Gutiérrez Pérez Gabriela G.
- Loeza Sebastián.
- Ramos García Luis Gerardo.
- Rico Mendoza Josué.

-----

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd

In [2]:
# Crear la ventana principal
root = tk.Tk()
root.title("Predicción de N-gramas")
root.geometry("700x600")

# Variables globales para almacenar el modelo y el tipo de n-grama seleccionado
bigramModel = pd.DataFrame()
trigramModel = pd.DataFrame()
ngramType = tk.StringVar()

# Función para cargar el CSV del modelo de bigramas
def loadBigramFile():
    filePath = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if filePath:
        try:
            global bigramModel
            bigramModel = pd.read_csv(filePath)
            messagebox.showinfo("Éxito", "Modelo de Bigramas cargado correctamente")
        except Exception as e:
            messagebox.showerror("Error", f"Error al cargar el archivo de Bigramas: {e}")

# Función para cargar el CSV del modelo de trigramas
def loadTrigramFile():
    filePath = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if filePath:
        try:
            global trigramModel
            trigramModel = pd.read_csv(filePath)
            messagebox.showinfo("Éxito", "Modelo de Trigramas cargado correctamente")
        except Exception as e:
            messagebox.showerror("Error", f"Error al cargar el archivo de Trigramas: {e}")

# Función para predecir la siguiente palabra basada en el modelo seleccionado
def predictNextWord(model, startWords, n=3):
    if len(startWords) == 1:
        bigrams = model[model['Term1'] == startWords[0]].sort_values(by='BigramProb', ascending=False)
        return bigrams.head(n)[['Term2', 'BigramProb']].values
    elif len(startWords) == 2:
        trigrams = model[(model['Term1'] == startWords[0]) & (model['Term2'] == startWords[1])].sort_values(by='TrigramProb', ascending=False)
        return trigrams.head(n)[['Term3', 'TrigramProb']].values
    return []

# Función para manejar la predicción y visualización de las opciones
def predictNextWordInterface():
    inputWords = predictiveEntry.get().split()
    if not inputWords:
        messagebox.showwarning("Advertencia", "Por favor ingrese una o dos palabras")
        return

    model = bigramModel if ngramType.get() == "Bigramas" else trigramModel
    
    if model.empty:
        messagebox.showerror("Error", "No se ha cargado un modelo.")
        return

    options = predictNextWord(model, inputWords)
    
    if options.size == 0:
        messagebox.showinfo("Predicción", "No se encontraron predicciones para las palabras ingresadas.")
    else:
        wordsList = [word for word, prob in options]
        wordsList.append('.')  # Añadir la opción de finalizar con un punto
        
        # Establecer opciones en el Combobox
        wordCombobox['values'] = wordsList
        wordCombobox.current(0)  # Seleccionar la primera opción

        # Si es la primera palabra, agregarla a `predictiveTextBox`
        if predictiveTextBox.get(1.0, tk.END).strip() == "":  # Verifica si está vacío
            predictiveTextBox.insert(tk.END, " ".join(inputWords))  # Añade la palabra inicial

# Función para seleccionar una palabra del combobox y continuar la predicción
def continueSentence():
    selectedWord = wordCombobox.get()
    
    if selectedWord == ".":
        currentText = predictiveTextBox.get(1.0, tk.END).strip()
        newText = f"{currentText} {selectedWord}".strip()  # Añadir el punto
        predictiveTextBox.delete(1.0, tk.END)  # Limpiar el cuadro de texto
        predictiveTextBox.insert(tk.END, newText)  # Actualizar con el texto completo
        messagebox.showinfo("Info", "Proceso finalizado.")
        predictiveEntry.delete(0, tk.END)
    
    elif selectedWord == "&":
        messagebox.showinfo("Info", "Proceso finalizado.")
        predictiveEntry.delete(0, tk.END)
        predictiveTextBox.delete(1.0, tk.END)
    
    else:
        # Obtener el texto completo actual del cuadro de texto
        currentText = predictiveTextBox.get(1.0, tk.END).strip()
        
        # Añadir la nueva palabra seleccionada al texto completo
        newText = f"{currentText} {selectedWord}".strip()  # Añadir la nueva palabra al historial
        predictiveTextBox.delete(1.0, tk.END)  # Limpiar el cuadro de texto
        predictiveTextBox.insert(tk.END, newText)  # Actualizar con el texto completo

        # Actualizar el cuadro de entrada para predecir con las últimas palabras según el tipo de n-grama
        predictiveEntry.delete(0, tk.END)
        if ngramType.get() == "Bigramas":
            # Si es bigrama, usa solo la última palabra
            predictiveEntry.insert(0, selectedWord)
        elif ngramType.get() == "Trigramas":
            # Si es trigramas, usa las últimas dos palabras
            lastWords = newText.split()[-2:]  # Obtiene las dos últimas palabras
            predictiveEntry.insert(0, " ".join(lastWords))

# Función para refrescar la entrada de palabras, el combobox y el cuadro de texto
def refreshOptions():
    predictiveEntry.delete(0, tk.END)
    predictiveTextBox.delete(1.0, tk.END)
    wordCombobox.set('')  # Limpiar el combobox

# Frame principal
frame = ttk.Frame(root, padding="10")
frame.pack(fill="both", expand=True)

# Frame para los botones de cargar CSV
load_frame = ttk.Frame(frame)
load_frame.pack(pady=10)

# Botones para cargar los archivos CSV
ttk.Button(load_frame, text="Cargar CSV de Bigramas", command=loadBigramFile).pack(side=tk.LEFT, padx=5)
ttk.Button(load_frame, text="Cargar CSV de Trigramas", command=loadTrigramFile).pack(side=tk.LEFT, padx=5)

# Selector de bigramas o trigramas
ngram_frame = ttk.Frame(frame)
ngram_frame.pack(pady=10)
ttk.Label(ngram_frame, text="Selecciona el tipo de n-grama:").pack(side=tk.LEFT)
ttk.Radiobutton(ngram_frame, text="Bigramas", variable=ngramType, value="Bigramas").pack(side=tk.LEFT, padx=5)
ttk.Radiobutton(ngram_frame, text="Trigramas", variable=ngramType, value="Trigramas").pack(side=tk.LEFT)

# Entrada para las palabras iniciales
ttk.Label(frame, text="Escribe una o dos palabras para predecir:").pack(pady=10)
predictiveEntry = ttk.Entry(frame)
predictiveEntry.pack(pady=10)

# Frame para el botón de predecir
predict_frame = ttk.Frame(frame)
predict_frame.pack(pady=10) 

# Botón para predecir la siguiente palabra
predictButton = ttk.Button(predict_frame, text="Predecir siguiente palabra", command=predictNextWordInterface)
predictButton.pack(pady=10)

# Menú desplegable para mostrar las opciones de predicción
ttk.Label(frame, text="Selecciona una palabra predicha:").pack(pady=10)
wordCombobox = ttk.Combobox(frame)
wordCombobox.pack(pady=10)

# Frame para el botón de seleccionar palabra
select_frame = ttk.Frame(frame)
select_frame.pack(pady=10)  

# Botón para seleccionar la palabra y continuar la oración
selectButton = ttk.Button(select_frame, text="Seleccionar palabra", command=continueSentence)
selectButton.pack(pady=10)

# Caja de texto 
ttk.Label(frame, text="Texto generado:").pack(pady=10)
predictiveTextBox = tk.Text(frame, height=3.5)
predictiveTextBox.pack(pady=10)

refreshButton = ttk.Button(frame, text="Refresh", command=refreshOptions)
refreshButton.pack(pady=10)

In [3]:
root.mainloop()

2024-11-05 20:31:50.242 python[21633:1922360] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-05 20:31:50.242 python[21633:1922360] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2024-11-05 20:32:01.519 python[21633:1922360] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/sebastianloeza/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/4w/0qtssn4s1_bck4zfcq9k819c0000gn/T/ipykernel_21633/1255404829.py", line 56, in predictNextWordInterface
    options = predictNextWord(model, inputWords)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/4w/0qtssn4s1_bck4zfcq9k819c0000gn/T/ipykernel_21633/1255404829.py", line 36, in predictNextWord
    bigrams = model[model['Term1'] == startWords[0]].sort_values(by='BigramProb', ascending